# Assignment 9

In [1]:
import numpy as np
import pandas as pd

### Part A

Consider a two period binomial model. Suppose after each period, the stock will either rise in value by ```r``` percent, or fall in value by ```f``` percent. Currently, the stock price is ```P```, and the risk free rate is ```rf```.

Write the function ```european_put```, which calculates the value of a one year European Put option with a strike price of ```K```. 

In [50]:
def european_put(P, K, r, f, rf):
    P_rise = ((((1+rf)**(1/2))-1)-(f))/(r-f)
    P_drop = 1-P_rise
    c1_i = P*(1+r)
    c1_d = P*(1+f)
    c2_ii = c1_i*(1+r)
    c2_0 = c1_d*(1+r)
    c2_dd = c1_d*(1+f)
    cii = max(0, K-c2_ii)
    c0 = max(0, K-c2_0)
    cdd = max(0, K-c2_dd)
    p1_r = (P_rise*cii + P_drop*c0)/((1+rf)**(1/2))
    p1_d = (P_rise*c0 + P_drop*cdd)/(1+rf)**(1/2)
    p0 = (P_rise*p1_r + P_drop*p1_d)/(1+rf)**(1/2)
    return p0


print(european_put(40,40, 0.25, -0.2, 0.1))
# european_put(40,40, 0.25, -0.2, 0.1) should return 2.616...

2.616751810884595


Write the function ```american_put```, which calculates the value of a one year American Put option with a strike price of ```K```. 

In [51]:
def american_put(P, K, r, f, rf):
    c1_d = max(0,K-P*(1+f))
    c1_i = max(0,K-P*(1+r))
    P_rise = ((((1+rf)**(1/2))-1)-(f))/(r-f)
    P_drop = 1-P_rise
    P0 = (P_rise*c1_i + P_drop*c1_d)/((1+rf)**(1/2))
    return P0

print(american_put(40,40, 0.25, -0.2, 0.1))
# american_put(40,40, 0.25, -0.2, 0.1) should return 3.410...

3.4102797610131605


### Part B

In addition to simple calls and puts, a wide variety of exotic options are traded in over-the-counter markets. In most cases, there are no analytic formulas available to value these options, and so numerical techniques must be used. One example of these techniques are Monte Carlo methods. These are particularly useful in certain contexts, such as when option payoffs depend not only on the stock price at expiration but also upon the history of the stock price sampled at various points during the life of the option. This question will explore some basic examples, and it will require relatively large number of paths. Assume that the stock price will be needed at the end of each of the next 12 months, in addition to S0 today, and start by simulating `N` such stock price paths. In particular:

•	You have been provided the function ```gen_random_vars```, which generates random variables that follow a standard normal distribution (Note that you should never actually call ```gen_random_vars``` in any of your functions, this is simply meant to help give you inputs to help test your code). This outputs a table with `N` rows and 12 columns. Denote the value of the random variable on path i for month j by zi,j where i = 1, . . . ,`N` and j = 1, . . . ,12.

Write the function ```create_paths```, which takes as an argument a DataFrame ```random_vars```, the output of the function ```gen_random_vars```, and ```S0```, the initial stock price.

This function will generate a DataFrame with `N` stock price paths $S_{i,j}$ where $i = 1, . . . ,N$ and $j = 0, . . . ,12$. In each case, $S_{i,0} =$ ```S_0```. For the remaining entries in this table, define $∆t = 1/12$ and set

$$ S_{i,j} = S_{i, j-1} \exp [(r - \sigma^2 / 2) \Delta t + \sigma \sqrt{\Delta t} z_{i,j}]$$

 
for i = 1, . . . ,`N` and j = 1, . . . ,12. (If you are wondering why we are using the risk-free rate r here instead of the expected rate of return on the underlying stock, it is because we are using risk-neutral valuation.) 

Note that throughout the rest of this question, $S_T$ on each simulated path i is defined to be equal to $S_{i,12}$.



In [52]:
def gen_random_vars(SEED, N):
    rng = np.random.default_rng(SEED)

    zArr = np.array([rng.normal(size=N)])
    for i in range(11):
        z = rng.normal(size=N)
        zArr = np.r_[zArr,[z]]
    

    randomVariables = pd.DataFrame(zArr.transpose())
    return randomVariables

r_vars = gen_random_vars(727, 10000)
r_vars

0         1         2         3         4         5         6   \
0     1.744763 -0.514250 -0.710679  1.495061  0.413633 -2.361391  0.122873   
1    -0.962309 -0.576888  0.396051  1.595821  0.008168 -0.348428 -0.340074   
2     0.496945  0.579497 -0.315875  0.330535 -0.045317  0.722330  0.760781   
3    -0.282490 -0.940921  1.175841  0.405341 -0.923402  0.977314 -0.128823   
4    -0.400287 -1.035328 -1.143469 -0.976739  0.649610 -0.414282 -0.257923   
...        ...       ...       ...       ...       ...       ...       ...   
9995  0.812176  0.492083  0.234801 -0.822796 -1.171159 -1.472986  0.598065   
9996 -0.714678 -0.710493 -0.644405  0.711664 -0.901012  0.526393 -2.218868   
9997 -1.684857  0.816217  0.048860  0.366472  0.296956 -0.250303  1.962326   
9998 -0.418933  0.839082 -2.012594  1.065439 -0.527060 -0.770415 -0.886027   
9999 -0.471454  0.268310  0.767433 -1.018053 -1.031914 -0.759433  0.806048   

            7         8         9         10        11  
0    -0.095099 -0.884679 -0.148495 -0.508705  1.562580  
1     0.230534 -0.556868 -1.567693 -0.009760  0.885091  
2    -0.070977  0.250281 -0.329167  0.407775  0.864023  
3    -0.688989 -1.411402  0.010026 -1.573026  0.730829  
4     0.762621 -1.664448 -1.159700  0.268761  1.822126  
...        ...       ...       ...       ...       ...  
9995 -0.239307 -0.822385  0.653382  0.729384  0.928624  
9996  2.422271  0.392027  1.610234  0.387898 -0.058958  
9997  1.416475  0.541789  0.660781  1.132385  0.190961  
9998  0.911825 -0.648514 -2.227339 -0.125608 -1.135578  
9999  0.445378  0.758105  0.378315 -0.898852  0.473172  

[10000 rows x 12 columns]

In [53]:
def create_paths(random_vars, S0, rf, sigma):
    total_col = len(random_vars)
    period = 12
    delta_t = 1 / 12
    paths = pd.DataFrame(index=range(total_col), columns=range(13))
    paths.iloc[:, 0] = S0
    for i in range(total_col):
        for j in range(1, period + 1):
            paths.iloc[i, j] = paths.iloc[i, j - 1] * np.exp(
                (rf - (sigma**2) / 2) * delta_t
                + sigma * np.sqrt(delta_t) * random_vars.iloc[i, j - 1]
            )
    return paths

# If we let sample_paths = create_paths(r_vars, 72.25, 0.04 , 0.35),
# The last column of the first row of sample_paths should equal 71.56...
sample_paths = create_paths(r_vars, 72.25, 0.04 , 0.35)
print(sample_paths.iloc[:,-1][0])
print(sample_paths)

/var/folders/2f/z74kz4310b52kl5mn52t9l8c0000gn/T/ipykernel_85005/168250157.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  paths.iloc[:, 0] = S0


71.56193284262598
         0          1          2          3          4          5          6   \
0     72.25  86.025679  81.525603  75.742687   87.93751  91.528239  71.972963   
1     72.25  65.439981  61.625511  64.028005  75.097372  75.026399  72.302976   
2     72.25  75.835839  80.266339  77.607549  80.101124  79.594133  85.468773   
3     72.25   70.09278  63.623431  71.522611  74.380734  67.635232  74.522593   
4     72.25  69.263496   62.27384  55.381117  50.088013  53.391152  51.111867   
...     ...        ...        ...        ...        ...        ...        ...   
9995  72.25  78.290065  82.135318   83.95834  77.124238  68.396198  58.834085   
9996  72.25  67.097921  62.339587   58.30673  62.542765  56.999633  60.006854   
9997  72.25  60.832839  65.945359  66.154461  68.528414  70.490711  68.608773   
9998  72.25   69.13313  75.116554  61.186409  68.020035  64.378472  59.451957   
9999  72.25  68.767247  70.531949  76.083662   68.52526  61.631362  56.978256   

         

•	Write the function ```payoff_table```, which returns another DataFrame with `N` rows and 6 columns. The columns will be used to store discounted payoffs at expiration for various different options, as indicated below:

–	European call option, with payoff $max(S_T − K,0)$.

–	European put option, with payoff $max(K - S_T ,0)$.

–	Average price put option, with payoff $max(K − S^-,0)$, where
$S¯ = S_0 + S_1 + · · · + S_{12}$ .

–	Floating lookback call option, with payoff $S_T - S_{min}$, where $S_{min}$ on each path is the minimum of the stock prices $S_0, S_1, . . . , S_{12}$ on that path.

–	Up-and-out call option with barrier ```B```, with payoff $max(S_T - K,0)$ as long as none of the observed monthly stock prices on the given path are greater than or equal to ```B```. If any of the monthly stock prices are greater than or equal to B, then the payoff is zero.

–	Up-and-in call option with barrier ```B``` , with payoff $max(S_T - K,0)$ as long as at least one of the observed monthly stock prices on the given path is greater than or equal to ```B```. If this condition is not satisfied, the option payoff is zero.

Note that “discounted payoffs” implies that the entries in this table should have the payoff functions multiplied by $e^{−rT}$ , where T = 1.

This function will also take in a dataframe ```paths```, which is the output from the previous function ```create_paths```.

In [54]:
def payoff_table(paths, K, T, rf, B):
    total = range(len(paths))
    answer = pd.DataFrame(
        index = total,
        columns = [
            "European Call",
            "European Put",
            "Average Price Put",
            "Floating Lookback",
            "Up-and-out",
            "Up-and-in",
        ],
    )
    rate = np.exp((0-rf) * T)
    for i in total:
        for j in range(1, T + 1):
            St = paths.iloc[i, 12]
            answer.loc[i, "European Call"] = max(St - K, 0)
            answer.loc[i, "European Put"] = max(K - St, 0)
            S_average = paths.iloc[i].mean()
            answer.loc[i, "Average Price Put"] = max(K - S_average, 0)
            S_min = paths.iloc[i].min()
            answer.loc[i, "Floating Lookback"] = max(St - S_min, 0)
            uo = (paths.iloc[i, j] >= B).any()
            answer.loc[i, "Up-and-out"] = (max(St - K, 0) if uo else 0)
            ui = (paths.iloc[i, j] >= B).any()
            answer.loc[i, "Up-and-in"] = max(St - K, 0) if not ui else 0
    answer = answer * rate
    return answer

            
# If we let sample_payoffs = payoff_table(sample_paths, 70, 1, 0.04, 90)
# The payoff of the first European Call should be 1.500...
sample_payoffs = payoff_table(sample_paths, 70, 1, 0.04, 90)
print(sample_payoffs.iloc[:,-1][0])
print(sample_payoffs)

1.5006885798602076
     European Call European Put Average Price Put Floating Lookback  \
0         1.500689          0.0               0.0           9.93743   
1              0.0     7.338567             2.899          5.028425   
2        31.017676          0.0               0.0           28.8559   
3              0.0    15.259282          3.797994          3.615473   
4              0.0    19.775742         16.035582          8.905278   
...            ...          ...               ...               ...   
9995      0.154346          0.0           1.20383         13.755086   
9996        6.4387          0.0          5.151621         27.700383   
9997     51.182324          0.0               0.0         59.990036   
9998           0.0    29.949076         10.683546               0.0   
9999           0.0     1.213008          2.458833         11.298147   

     Up-and-out  Up-and-in  
0           0.0   1.500689  
1           0.0        0.0  
2           0.0  31.017676  
3           

Write the function ```statistics```, which takes as input a DataFrame ```payoffs```, formatted as one produced by ```payoff_table```, and displays the following statistics for each of the options above (below N is the number of simulated paths):

•	Mean discounted payoff µ. This is the estimated option premium.

•	Standard error of option premium ϕ. This is the standard deviation of the option payoff divided by √N .

•	Lower bound for 95% confidence interval for option premium, which is µ - 1.96ϕ.

•	Upper bound for 95% confidence interval for option premium, which is µ + 1.96ϕ.

The size of the returned DataFrame should have 6 rows and 4 columns. 

In [55]:
def statistics(payoffs):
    mean_payoff = payoffs.mean().tolist()
    N = len(mean_payoff)
    std = (payoffs.std() / np.sqrt(len(payoffs))).tolist()
    upper = []
    lower = []
    for i in range(N):
        upper.append(mean_payoff[i] + 1.96 * std[i])
        lower.append(mean_payoff[i] - 1.96 * std[i])
    output = pd.DataFrame(data = {
        "Mean" : mean_payoff,
        "Standard Error" : std,
        "Lower Bound" : lower,
        "Upper Bound" : upper
    }).rename(index={0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}, errors="raise")
    return output

# If we call statistics(sample_payoffs), the mean of the European Puts should be 7.370...
print(statistics(sample_payoffs))

                        Mean  Standard Error  Lower Bound  Upper Bound
European Call      12.093962        0.190808    11.719979    12.467945
European Put        7.370213        0.102129     7.170040     7.570387
Average Price Put   3.833641        0.058521     3.718940     3.948342
Floating Lookback  15.992686        0.184633    15.630806    16.354566
Up-and-out          0.441739        0.048119     0.347425     0.536053
Up-and-in          11.652223        0.187408    11.284904    12.019542


One disadvantage of Monte Carlo methods is that they can be slow to converge. On the other hand, there are situations where they are the only methods that can be used. As a result, there has been a lot of research into variance reduction methods, in an effort to improve the rate of convergence. 

One of the simplest such approaches is called antithetic variates. This relies on the fact that if Z is a standard normal random variable, then so is -Z. To see how this can be used, generate another `N` stock price paths, but this time using $-z_{i,j}$ instead of $z_{i,j}$ throughout.

In order to accomplish this, write the function ```create_negative_paths```. This function has the same arguments as ```create_paths```, but creates the stock prices in the manner described above.

In [56]:
def create_negative_paths(random_vars, S0, rf, sigma):
    total_col = len(random_vars)
    period = 12
    delta_t = 1 / 12
    paths = pd.DataFrame(index=range(total_col), columns=range(13))
    paths.iloc[:, 0] = S0 
    for i in range(total_col):
        for j in range(1, period + 1):
            paths.iloc[i, j] = paths.iloc[i, j - 1] * np.exp(
                (rf - (sigma**2) / 2) * delta_t
                + sigma * np.sqrt(delta_t) * (0-(random_vars.iloc[i, j - 1]))
            )
    return paths
sample_paths_negative = create_negative_paths(r_vars, 72.25, 0.04 , 0.35)
print(sample_paths_negative.iloc[:,-1][0])

/var/folders/2f/z74kz4310b52kl5mn52t9l8c0000gn/T/ipykernel_85005/3840216710.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  paths.iloc[:, 0] = S0


69.90948861632032


Write the function ```antithetic_statistics```.

In this function, you will use ```payoff_table``` to create two tables, one using paths generated from ```create_paths```, and one generated from ```create_negative_paths```.

Next, create a third table which contains the average payoff between the two tables.

Finally, call ```statistics``` on this third table. Return the table produced by ```statistics```.

If you compare this table with the output of `statistics` (as above, without creating two tables), you should notice that Mean, Lower Bound, and Upper Bound columns should have relatively similar values, while the Standard Error column has decreased significantly.

In [57]:
def antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    cp = create_paths(random_vars, S0, rf, sigma)
    cp_negative = create_negative_paths(random_vars, S0, rf, sigma)
    payoff_cp = payoff_table(cp, K, T, rf, B)
    payoff_cp_negative = payoff_table(cp_negative, K, T, rf, B)
    final = (payoff_cp + payoff_cp_negative) / 2
    answer = statistics(final)
    return answer
antithetic_statistics(r_vars,72.25,0.04,0.35,70,1,90)

/var/folders/2f/z74kz4310b52kl5mn52t9l8c0000gn/T/ipykernel_85005/168250157.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  paths.iloc[:, 0] = S0
/var/folders/2f/z74kz4310b52kl5mn52t9l8c0000gn/T/ipykernel_85005/3840216710.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  paths.iloc[:, 0] = S0


Mean  Standard Error  Lower Bound  Upper Bound
European Call      12.195273        0.105566    11.988364    12.402182
European Put        7.330247        0.049657     7.232919     7.427576
Average Price Put   3.808618        0.031072     3.747716     3.869519
Floating Lookback  16.097769        0.089103    15.923127    16.272410
Up-and-out          0.440744        0.035459     0.371244     0.510244
Up-and-in          11.754529        0.104513    11.549683    11.959375